# --- Day 14: Regolith Reservoir ---



In [257]:
import numpy as np


def pretty_print_cave(mat_cave):
    for i in range(0, len(mat_cave)):
        print(
            "".join(
                [
                    "." if y == 0 else "+" if y == -1 else "o" if y == 2 else "#"
                    for y in mat_cave[i]
                ]
            )
        )
    return 0


def read_cave_data(filename, display=False):
    # Read cave input data and set up matrix
    # with cave layout
    cave_lines = []

    with open(filename) as infile:
        for line in infile:
            cave_line = [
                tuple(map(int, x.split(","))) for x in line.strip().split(" -> ")
            ]
            cave_lines.append(cave_line)

    # Get dimensions of cave
    cave_dim_xmin = min([min([y[0] for y in x]) for x in cave_lines])
    cave_dim_xmax = max([max([y[0] for y in x]) for x in cave_lines])
    cave_dim_ymin = min([min([y[1] for y in x]) for x in cave_lines])
    cave_dim_ymax = max([max([y[1] for y in x]) for x in cave_lines])
    if cave_dim_ymin <= 0:
        raise ValueError(
            f"Unexpected cave dimension cave_dim_ymin: {cave_dim_ymin} (expect > 0)"
        )

    # Set up matrix to represent cave - add margin
    # of 1 to each horizontal side, margin of 3 to bottom
    matrix_cave = np.full((cave_dim_ymax + 4, (cave_dim_xmax - cave_dim_xmin) + 3), 0)

    # Loop over traces in scan to build cave
    for cave_line in cave_lines:
        for i in range(0, len(cave_line) - 1):
            x_start = min(cave_line[i][0], cave_line[i + 1][0]) - (cave_dim_xmin - 1)
            x_end = max(cave_line[i][0], cave_line[i + 1][0]) - (cave_dim_xmin - 2)
            y_start = min(cave_line[i][1], cave_line[i + 1][1])
            y_end = max(cave_line[i][1], cave_line[i + 1][1]) + 1
            matrix_cave[y_start:y_end, x_start:x_end] = 1

    # Starting position relative to matrix indices
    start_loc = (0, 500 - (cave_dim_xmin - 1))
    matrix_cave[start_loc] = -1

    if display:
        pretty_print_cave(matrix_cave)

    return matrix_cave, start_loc


read_cave_data("inputs/day14-example.txt", True);


.......+....
............
............
............
.....#...##.
.....#...#..
...###...#..
.........#..
.........#..
.#########..
............
............
............


In [255]:
def simulate_sand(matrix_cave, start_loc, print_grains=[]):

    # One grain of sand at a time
    num_grains = 1
    grain_out_of_bounds = False
    while not grain_out_of_bounds:
        grain_at_rest = False
        grain_position = start_loc

        # Update position
        while not grain_at_rest:
            grain_out_of_bounds = not any(
                matrix_cave[grain_position[0] :, grain_position[1]] > 0
            )
            if not grain_out_of_bounds:
                grain_position = (
                    grain_position[0]
                    + np.where(matrix_cave[grain_position[0] :, grain_position[1]] > 0)[
                        0
                    ][0]
                    - 1,
                    grain_position[1],
                )
                if (
                    sum(
                        matrix_cave[
                            grain_position[0] + 1,
                            grain_position[1] - 1 : grain_position[1] + 2,
                        ]
                        > 0
                    )
                    == 3
                ):
                    grain_at_rest = True
                elif matrix_cave[grain_position[0] + 1, grain_position[1] - 1] == 0:
                    grain_position = (grain_position[0] + 1, grain_position[1] - 1)
                else:
                    grain_position = (grain_position[0] + 1, grain_position[1] + 1)
            else:
                grain_at_rest = True
        if grain_position == start_loc:
            grain_out_of_bounds = True
            num_grains += 1
        num_grains += 1
        matrix_cave[grain_position] = 2

        if num_grains in print_grains:
            print(f"\nGrain: {num_grains}")
            pretty_print_cave(matrix_cave)

    return num_grains - 2


mat_cave, start_pos = read_cave_data("inputs/day14.txt")
simulate_sand(mat_cave.copy(), start_pos, [])


683

# --- Part Two ---



In [256]:
def part_two(matrix_cave, start_loc, print_grains=[]):
    # Adjust cave matrix input
    floor_height = max(np.where(matrix_cave == 1)[0]) + 2
    matrix_cave = np.pad(mat_cave, [(0, 0), (floor_height, floor_height)])
    matrix_cave[floor_height, :] = 1
    start_loc = (start_loc[0], start_loc[1] + floor_height)
    return simulate_sand(matrix_cave, start_loc, print_grains)


part_two(mat_cave.copy(), start_pos)


28821